In [1]:
import pandas as pd
import tensorflow as tf
import sys
sys.path.insert(1,"../../")
from NeuralCore import *

c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\deves\appd

In [2]:
stock = "AMBUJACEM"
options = []
prices = []
errors = []

for i in range(12):
    if i+1 not in errors:
        options.append("%s-options-CE-%s.csv"%(stock,str(i+1)))
        prices.append("%s-prices-%s.csv"%(stock,str(i+1)))

In [3]:
# opt = pd.read_csv(options[0])
# price = pd.read_csv(options[0])

opts = []
for x in options:
    tmp = pd.read_csv(x)
    opts.append(tmp)
    
opt = pd.concat(opts)
    
price = []
for x in prices:
    tmp = pd.read_csv(x)
    price.append(tmp)

price = pd.concat(price)
price = price[["Date","Close","Open"]]

In [4]:
opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Premium Turnover in Lacs,Open Int,Change in OI,Underlying Value
0,AMBUJACEM,01-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,10.75,10.75,10.70,0,0.00,0.00,5000,0,197
1,AMBUJACEM,02-Jan-2020,30-Jan-2020,CE,190,12.95,12.95,12.95,12.95,12.95,18.00,1,5.07,0.32,5000,0,205.35
2,AMBUJACEM,03-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,12.95,12.95,16.20,0,0.00,0.00,5000,0,203.4
3,AMBUJACEM,06-Jan-2020,30-Jan-2020,CE,190,11.35,11.35,11.35,11.35,11.35,11.35,1,5.03,0.28,7500,2500,196.9
4,AMBUJACEM,07-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,11.35,11.35,12.55,0,0.00,0.00,7500,0,199


In [5]:
opt = opt.drop(["No. of contracts","LTP","Settle Price","Premium Turnover in Lacs","Underlying Value"],axis=1)

In [6]:
opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,Turnover in Lacs,Open Int,Change in OI
0,AMBUJACEM,01-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,10.75,0.00,5000,0
1,AMBUJACEM,02-Jan-2020,30-Jan-2020,CE,190,12.95,12.95,12.95,12.95,5.07,5000,0
2,AMBUJACEM,03-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,12.95,0.00,5000,0
3,AMBUJACEM,06-Jan-2020,30-Jan-2020,CE,190,11.35,11.35,11.35,11.35,5.03,7500,2500
4,AMBUJACEM,07-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,11.35,0.00,7500,0


In [7]:
opt["Change"] = (opt["Close"] - opt["Open"])*100/opt["Open"]
opt["Theta"] = pd.to_datetime(opt["Expiry"]) - pd.to_datetime(opt["Date"])
opt["Theta"] = opt["Theta"].dt.days.astype('int16')
opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,Turnover in Lacs,Open Int,Change in OI,Change,Theta
0,AMBUJACEM,01-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,10.75,0.00,5000,0,inf,29
1,AMBUJACEM,02-Jan-2020,30-Jan-2020,CE,190,12.95,12.95,12.95,12.95,5.07,5000,0,0.0,28
2,AMBUJACEM,03-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,12.95,0.00,5000,0,inf,27
3,AMBUJACEM,06-Jan-2020,30-Jan-2020,CE,190,11.35,11.35,11.35,11.35,5.03,7500,2500,0.0,24
4,AMBUJACEM,07-Jan-2020,30-Jan-2020,CE,190,0.00,0.00,0.00,11.35,0.00,7500,0,inf,23


In [8]:
opt.set_index("Date",inplace=True)
price.set_index("Date",inplace=True)

In [9]:
opt["OV"] = ((price["Close"] - opt["Strike Price"]) / opt["Close"]) * opt["Theta"]
opt["OV-Mean"] = (opt.OV.shift(1)+opt.OV.shift(2)+opt.OV.shift(3)) / 3
opt["Ratio"] = opt["OV"] / opt["OV-Mean"]
opt.loc[opt.Ratio <= 0.3,["buy"]] = 1

In [10]:
opt.fillna(0,inplace=True)
tmp = opt['Ratio']
tmp = np.array(tmp)
tmp2 = opt['buy']
tmp2 = np.array(tmp2)

In [11]:
opt.head()

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Turnover in Lacs,Open Int,Change in OI,Change,Theta,OV,OV-Mean,Ratio,buy
Date,,,,,,,,,,,,,,,,,
01-Jan-2020,AMBUJACEM,30-Jan-2020,CE,190,0.00,0.00,0.00,10.75,0.00,5000,0,inf,29,18.883721,0.000000,0.00000,0.0
02-Jan-2020,AMBUJACEM,30-Jan-2020,CE,190,12.95,12.95,12.95,12.95,5.07,5000,0,0.0,28,33.189189,0.000000,0.00000,0.0
03-Jan-2020,AMBUJACEM,30-Jan-2020,CE,190,0.00,0.00,0.00,12.95,0.00,5000,0,inf,27,27.938224,0.000000,0.00000,0.0
06-Jan-2020,AMBUJACEM,30-Jan-2020,CE,190,11.35,11.35,11.35,11.35,5.03,7500,2500,0.0,24,14.590308,26.670378,0.54706,0.0
07-Jan-2020,AMBUJACEM,30-Jan-2020,CE,190,0.00,0.00,0.00,11.35,0.00,7500,0,inf,23,18.237885,25.239240,0.72260,0.0


In [12]:
df = pd.concat(opts)
df["Ratio"] = tmp
df["buy"] = tmp2
x_data = df[["Ratio"]]
y_data = df["buy"]

In [13]:
import numpy as np

In [14]:
ratios = tf.feature_column.numeric_column("Ratio")
feature_colums = [ratios]

In [15]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.3)

In [18]:
train_fn = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=1,
                                              num_epochs=1000,shuffle=True)

In [19]:
model = tf.estimator.DNNClassifier(feature_columns=feature_colums,n_classes=2,
                                  hidden_units=[20,20,20,32,34,12,32,43,43])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\deves\\AppData\\Local\\Temp\\tmppb8t0g55', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016E77AB8710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
results = model.train(input_fn=train_fn,steps=10000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\deves\AppData\Local\Temp\tmppb8t0g55\model.ckpt.
INFO:tensorflow:loss = 0.6835227, step = 1
INFO:tensorflow:global_step/sec: 336.79
INFO:tensorflow:loss = 0.18229721, step = 101 (0.301 sec)
INFO:tensorflow:global_step/sec: 1165.91
INFO:tensorflow:loss = 6.2677817e-15, step = 201 (0.084 sec)
INFO:tensorflow:global_step/sec: 1412.21
INFO:t

INFO:tensorflow:global_step/sec: 1222.77
INFO:tensorflow:loss = 0.021355452, step = 6201 (0.082 sec)
INFO:tensorflow:global_step/sec: 1012.8
INFO:tensorflow:loss = 0.018691523, step = 6301 (0.099 sec)
INFO:tensorflow:global_step/sec: 1033.68
INFO:tensorflow:loss = 0.013430828, step = 6401 (0.096 sec)
INFO:tensorflow:global_step/sec: 1055.45
INFO:tensorflow:loss = 0.010495766, step = 6501 (0.096 sec)
INFO:tensorflow:global_step/sec: 1114.08
INFO:tensorflow:loss = 1.5242481e-11, step = 6601 (0.089 sec)
INFO:tensorflow:global_step/sec: 1208.03
INFO:tensorflow:loss = 0.016819727, step = 6701 (0.083 sec)
INFO:tensorflow:global_step/sec: 1101.83
INFO:tensorflow:loss = 0.0, step = 6801 (0.093 sec)
INFO:tensorflow:global_step/sec: 887.332
INFO:tensorflow:loss = 0.036058065, step = 6901 (0.113 sec)
INFO:tensorflow:global_step/sec: 1114.08
INFO:tensorflow:loss = 0.0004758834, step = 7001 (0.088 sec)
INFO:tensorflow:global_step/sec: 1152.49
INFO:tensorflow:loss = 6.604664e-10, step = 7101 (0.087 

In [21]:
evalute_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,num_epochs=1,
                                                        shuffle=False,batch_size=1)

In [22]:
model.evaluate(evalute_input_func)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-09T12:56:48Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\deves\AppData\Local\Temp\tmppb8t0g55\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-07-09-12:56:49
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9714286, accuracy_baseline = 0.78571427, auc = 0.9854545, auc_precision_recall = 0.950245, average_loss = 0.107839815, global_step = 10000, label/mean = 0.21428572, loss = 0.107839815, precision = 0.88235295, prediction/mean = 0.2351823, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\deves\AppData\Local\Te

{'accuracy': 0.9714286,
 'accuracy_baseline': 0.78571427,
 'auc': 0.9854545,
 'auc_precision_recall': 0.950245,
 'average_loss': 0.107839815,
 'label/mean': 0.21428572,
 'loss': 0.107839815,
 'precision': 0.88235295,
 'prediction/mean': 0.2351823,
 'recall': 1.0,
 'global_step': 10000}

In [23]:
test = {"Ratio":[0.4,0.2,-0.4,6]}

In [24]:
df = pd.DataFrame(test)

In [25]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=df,num_epochs=1,shuffle=False,batch_size=5)

In [26]:
a = model.predict(predict_input_func)

In [27]:
t = list(a)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\deves\AppData\Local\Temp\tmppb8t0g55\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [28]:
t[1]

{'logits': array([3.6254437], dtype=float32),
 'logistic': array([0.9740538], dtype=float32),
 'probabilities': array([0.02594614, 0.9740538 ], dtype=float32),
 'class_ids': array([1], dtype=int64),
 'classes': array([b'1'], dtype=object)}

In [16]:
fts = ["Ratio"]
core = NeuralCore(x_data,y_data,features=fts)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\deves\\AppData\\Local\\Temp\\tmppkk0janf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B4EC68B3C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
core.train()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\deves\AppData\Local\Temp\tmppkk0janf\model.ckpt.
INFO:tensorflow:loss = 0.5269452, step = 1
INFO:tensorflow:global_step/sec: 778.625
INFO:tensorflow:loss = 1.1669049, step = 101 (0.130 sec)
INFO:tensorflow:global_step/sec: 1453.14
INFO:tensorflow:loss = 1.1209228, step = 201 (0.068 sec)
INFO:tensorflow:global_step/sec: 1432.39
INFO:tenso

In [18]:
core.evaluate()

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-09T13:01:54Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\deves\AppData\Local\Temp\tmppkk0janf\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-07-09-13:01:54
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9877301, accuracy_baseline = 0.80981594, auc = 0.99694526, auc_precision_recall = 0.98466945, average_loss = 0.050770875, global_step = 1000, label/mean = 0.19018404, loss = 0.050770875, precision = 0.9677419, prediction/mean = 0.19294469, recall = 0.9677419
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\deves\AppData\Loc